In [90]:
%load_ext autoreload
%autoreload 2
from diacritization import recodex_predict
from diacritization import Dataset
import numpy as np
import lzma
import pickle
from diacritization import flatten
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [92]:
dataset = Dataset()

In [95]:
data = dataset.data.translate(Dataset.DIA_TO_NODIA)

In [98]:
from diacritization import recodex_predict

In [99]:
recodex_predict(data)

'Měla na ruce nejkrásnějši náramek , jaký jsem kdy viděl - secesní víla se na něm proplétala mezi brilianty a smaragdy .\nTenhle náramek měl cenu luxusního auta , jenže jeho krása byla ještě větší , byl to jeden z těch předmětů , na které člověk pohlédne a vzdává chválu Stvořiteli , který dal lidem takové nadání .\nStará dáma si všimla , že se na náramek dívám .\n" Líbí se vám ? "\n" Hezčí šperk jsem ještě neviděl , " řekl jsem .\n" Snad si říkáte - ta bude mít hodně pěkných šperků , " usmála se na mě .\n" Ale tohle je jediný cenný klenot , co mám . "\nOdepjala náramek a položila ho na stůl .\nVšimla jsem si , že pokoj je zaplněný květinami , zřejmě k téhle dívce každý nějaké přinesl - a podle toho k ní musela chodit spousta lidí .\nDaly jsme se do řeči .\nJestli si představujete , že mluvila nějakým kazatelským tónem nebo vemlouvavým hlasem , kdepak .\nMluvila , jako kdyby byla mou kamarádkou z diskotéky , smála se každou chvíli - to bylo zvláštní , stále vyhledávala nějaký veselý rys

In [100]:
len(_)

563361

In [102]:
len(data)

563361

In [84]:
with lzma.open("diacritization.model", "rb") as model_file:
    nn = pickle.load(model_file)

with lzma.open("onehot.encoder", "rb") as model_file:
    ohe = pickle.load(model_file)

with lzma.open("label.encoder", "rb") as model_file:
    le = pickle.load(model_file)

def find_ngrams(s, n):
    return zip(*[s[i:] for i in range(n)])

input_len = 13
features = np.concatenate([
        np.array(
            list(
                find_ngrams(
                    "".join(["#" for i in range(int((input_len - 1) / 2))])
                    + sentence.lower()
                    + "".join(["#" for i in range(int((input_len - 1) / 2))]),
                    input_len,
                )
            )
        )
        for sentence in data.data.translate(Dataset.DIA_TO_NODIA).split("\n")[:-1]
    ])

def convert_predictions(predictions,orig_text):
    predictions = "".join(
            np.where(
                ((np.array(list(predictions)) == " ")
                & (np.array(list(orig_text)) != " ")),
                np.array(list(orig_text)),
                np.array(list(predictions)),
            )
        )
    predictions = "".join(
            np.where(
                ((np.array(list(predictions)) != " ")
                & (np.array(list(orig_text)) == " ")),
                np.array(list(orig_text)),
                np.array(list(predictions)),
            )
        )
    result = [
        predicted_char.upper() if orig_char.isupper() else predicted_char for predicted_char,orig_char in zip(predictions,orig_text)
    ]
    return "".join(result)

def add_newlines(predictions,data):
    new_predictions = ""
    data_index = 0
    pred_index = 0
    while pred_index < len(predictions):
        if data[data_index] == "\n":
            new_predictions += "\n"
            data_index +=1 
        new_predictions += predictions[pred_index]
        pred_index +=1 
        data_index +=1
    return new_predictions + '\n'

orig_text = "".join(data.data.translate(Dataset.DIA_TO_NODIA).split("\n"))

features = ohe.transform(features)
predictions = nn.predict(features)
predictions = le.inverse_transform(predictions)
predictions = "".join(predictions)

predictions = convert_predictions(predictions,orig_text)

result = add_newlines(predictions,data.data)

In [88]:
with open("gold.txt", "w",encoding="utf-8") as text_file:
    text_file.write(data.data)

In [89]:
with open("pred.txt", "w",encoding="utf-8") as text_file:
    text_file.write(result)